# Generate volumetric slices in nrrd format

In [2]:
import voxcell
from voxcell import CellCollection, VoxelData, RegionMap
# from brainbuilder.utils import bbp
import numpy as np
import requests
from matplotlib import cm
import matplotlib
import pandas as pd
import numpy.linalg as la
from bluepy.enums import Cell
from bluepy import Circuit
from bluepy.geometry.roi import Cube
from coordinate_query import CoordinateQuery, query_enriched_positions, LON, TRA, RAD
import math
from log_progress import log_progress
from pathlib import Path
from voxcell.nexus.voxelbrain import Atlas
from bluepy import Circuit
from coordinate_query import CoordinateQuery, enriched_cells_positions, query_enriched_positions, LON, TRA, RAD
from log_progress import log_progress

In [3]:
circ_path = '/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/CircuitConfig'
slice_thickness = 400 # um
usertarget_path = f'slices{slice_thickness}.target'
circuit = Circuit(circ_path)

In [4]:
ATLAS_DIR = Path("/gpfs/bbp.cscs.ch/project/proj112/entities/atlas/20211004_BioM/")
CIRCUIT_DIR = Path('/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/')
c = Circuit((CIRCUIT_DIR / 'CircuitConfig').as_posix())
atlas = Atlas.open(ATLAS_DIR.as_posix())
orientation = atlas.load_data("orientation")

In [5]:
def update_user_target(filename, gids, target_name, notes):
    with open(filename, 'a') as f:
        f.write(notes)
        f.write('\nTarget Cell %s\n{' % target_name)
        f.write(' '.join(['a%d' % gid for gid in gids]))
        f.write('}\n\n')

def get_gids_in_voxel(mask,circuit):
    in_target = mask.lookup(circuit.cells.get(properties=list("xyz")).values)
    gids_in_target = circuit.cells.get().index.values[np.where(in_target==1)]
    return gids_in_target

In [6]:
# load the atlas in the coordinatequery
file_path = '/gpfs/bbp.cscs.ch/project/proj112/entities/atlas/20211004_BioM/coordinates.nrrd'
from coordinate_query import CoordinateQuery, query_enriched_positions, LON, TRA, RAD

q = CoordinateQuery(file_path)
# Enriched your ltr positions in the positions df from bluepy
xyz_ltr = enriched_cells_positions(circuit, q)
tot_distance = q.get_long_center_distance(q.min_max[LON][0], q.min_max[LON][1])
nslices = int(tot_distance/slice_thickness)
print('tot distance:', tot_distance, '# slices:', nslices)

tot distance: 13857.729952557209 # slices: 34


In [10]:
orientation.raw.shape

(308, 495, 464, 4)

In [13]:
atlas = Atlas.open(ATLAS_DIR.as_posix())
orientation = atlas.load_data("orientation")
coords = atlas.load_data('coordinates')

In [17]:
c.cells.get(12321)

etype                                                                     cACpyr
layer                                                                         SP
me_combo                       CA1_pyr_cACpyr_oh140807_A0_idB_2019030510521_S...
model_template                  hoc:CA1_pyr_cACpyr_oh140807_A0_idB_2019030510521
model_type                                                           biophysical
morph_class                                                                  PYR
morphology                     dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC_-...
mtype                                                                      SP_PC
orientation_w                                                           0.984479
orientation_x                                                          -0.164185
orientation_y                                                                0.0
orientation_z                                                          -0.062009
region                      

In [18]:
sample_sppc = c.cells.get(12321)

In [25]:
sample_sppc_index = q.xyz_to_idx([sample_sppc.x,sample_sppc.y,sample_sppc.z])[0]

In [27]:
sample_sppc_index

array([163,  44, 314])

In [34]:
orientation.raw[163,44,314,:]

array([ 0.9844787 , -0.16418463,  0.        , -0.06200899], dtype=float32)

## Method 1
Slice CA1 from tip to tip in desired slice thickness and then select one slice. Better for simulation campaings

In [14]:
print(f"slice thickness: {slice_thickness}")
slice_id = 1
for l in log_progress(range(0, int(q.get_long_center_distance(q.min_max[LON][0], q.min_max[LON][1])), slice_thickness), every=1):
    l_min, l_max = q.long_micro_meter_slice(l, slice_thickness) # relative coordinate
    percentage = (l_min + l_max)/2
    print(f"l:{l}, l_min:{l_min}, l_max:{l_max}, percentage={percentage}")
    cur_slice_mask = q._get_range_mask(min_value=l_min, max_value=l_max, axis=LON)
    orientation.with_data(np.asarray(cur_slice_mask, dtype=np.uint8)).save_nrrd(f"nrrd_masks_serially_generated_but_not_used/slice{slice_thickness}_{slice_id}.nrrd")
    slice_id += 1

slice thickness: 400


l:0, l_min:0.043989644635413404, l_max:0.07021988810231852, percentage=0.05710476636886597
l:400, l_min:0.07021988810231852, l_max:0.09645013156922366, percentage=0.08333500983577109
l:800, l_min:0.09645013156922366, l_max:0.12268037503612877, percentage=0.10956525330267622
l:1200, l_min:0.1226803750361288, l_max:0.14891061850303391, percentage=0.13579549676958136
l:1600, l_min:0.14891061850303391, l_max:0.17514086196993903, percentage=0.16202574023648647
l:2000, l_min:0.17514086196993905, l_max:0.20137110543684417, percentage=0.1882559837033916
l:2400, l_min:0.20137110543684417, l_max:0.22760134890374928, percentage=0.21448622717029672
l:2800, l_min:0.2276013489037493, l_max:0.25383159237065445, percentage=0.24071647063720186
l:3200, l_min:0.2538315923706544, l_max:0.28006183583755956, percentage=0.266946714104107
l:3600, l_min:0.28006183583755956, l_max:0.3062920793044647, percentage=0.29317695757101214
l:4000, l_min:0.30629207930446467, l_max:0.33252232277136984, percentage=0.319407

In [10]:
cur_slice_mask.shape

(308, 495, 464)

### selecting 25% percentile (dorsal side)

In [6]:
selected_l_min = 0.2276013489037493
selected_l_max = 0.25383159237065445
slice_mask_temp = q._get_range_mask(min_value=selected_l_min, max_value=selected_l_max, axis=LON)

In [7]:
slice_mask_temp.shape

(308, 495, 464)

### Extract layer nrrds

In [30]:
for layer_name in ['SO','SP','SR','SLM']:
    cur_layer_mask = atlas.get_region_mask(layer_name)
    layer_intersection_mask = np.logical_and(slice_mask_temp,cur_layer_mask.raw)
    orientation.with_data(np.asarray(layer_intersection_mask, dtype=np.uint8)).save_nrrd(f"slice{slice_thickness}_{layer_name}.nrrd")

### Saving the slice in nrrd format

In [8]:
slice_nrrd_file = 'slice400_method1.nrrd'
orientation.with_data(np.asarray(slice_mask_temp, dtype=np.uint8)).save_nrrd(slice_nrrd_file)

### Generate target gids from mask and update target file

In [9]:
int_mask = Atlas.open('.').load_data(slice_nrrd_file[:-5])
mask_gids = get_gids_in_voxel(int_mask,c)

In [33]:
notes = '# thickness = ' + str(slice_thickness) + 'um, longitudinal range: ' + str(np.round(selected_l_min,3)) + '-' + str(np.round(selected_l_max,3))
target_name = 'slice400'
#update_user_target(CIRCUIT_DIR.as_posix() + '/user.target', mask_gids, target_name, notes)
update_user_target(usertarget_path, mask_gids, target_name, notes)

# method 2

Directly select the longit position and thickness and extract 1 slice

In [22]:
total_center_longit_distance = q.get_long_center_distance(q.min_max[LON][0], q.min_max[LON][1])
slice_position_long_percentage = 0.25
slice_position_long = total_center_longit_distance * slice_position_long_percentage
slice_thickness_um = 400
slice_boundaries_lon = q.long_micro_meter_slice(slice_position_long,slice_thickness_um)

boundaries along longit axis are

In [23]:
slice_boundaries_lon

(0.2711719137317879, 0.29740215719869306)

In [24]:
slice_mask = q._get_range_mask(min_value=slice_boundaries_lon[0], max_value=slice_boundaries_lon[1], axis=LON)

saving as nrrd

In [25]:
orientation.with_data(np.asarray(slice_mask, dtype=np.uint8)).save_nrrd(f'slice400_method2.nrrd')

### Generate target gids from mask and update target file

In [28]:
int_mask = Atlas.open('.').load_data(slice_nrrd_file[:-5])
mask_gids = get_gids_in_voxel(int_mask,c)

In [29]:
notes = '# thickness = ' + str(slice_thickness) + 'um, longitudinal range: ' + str(np.round(slice_boundaries_lon[0],3)) + '-' + str(np.round(slice_boundaries_lon[1],3))
target_name = 'slice400_method2'
#update_user_target(CIRCUIT_DIR.as_posix() + '/user.target', mask_gids, target_name, notes)
update_user_target(usertarget_path, mask_gids, target_name, notes)

# Validation 

After generating slices, measure the distance again

In [26]:
q.get_long_center_distance(slice_boundaries_lon[0],slice_boundaries_lon[1])

347.3433299154493

In [27]:
# method 1 : slicegen approach euclidian thickness from middle
q.get_long_center_distance(selected_l_min,selected_l_max)

382.85694801434613

## Warning: Measurements are prone to curvature

As you can see from different slices along longit axis below , although the thicknesses given are the same, the measured distance can vary due to the:
- measuring method
- measuring points
- curvature changes along longit axis of CA1

In [119]:
for i in np.arange(0,1,0.1):
    total_center_longit_distance = q.get_long_center_distance(q.min_max[LON][0], q.min_max[LON][1])
    slice_position_long_percentage = i
    slice_position_long = total_center_longit_distance * slice_position_long_percentage
    slice_thickness_um = 400
    slice_boundaries_lon = q.long_micro_meter_slice(slice_position_long,slice_thickness_um)
    print(f"Longit percentage {i}: Measured Central Dist = {q.get_long_center_distance(slice_boundaries_lon[0],slice_boundaries_lon[1])}")

Longit percentage 0.0: Measured Central Dist = 472.74517448621305
Longit percentage 0.1: Measured Central Dist = 272.62415558157835
Longit percentage 0.2: Measured Central Dist = 358.18775056957213
Longit percentage 0.30000000000000004: Measured Central Dist = 280.23462108065553
Longit percentage 0.4: Measured Central Dist = 282.9600697769798
Longit percentage 0.5: Measured Central Dist = 338.8726568463918
Longit percentage 0.6000000000000001: Measured Central Dist = 373.5158946560844
Longit percentage 0.7000000000000001: Measured Central Dist = 422.04620965061434
Longit percentage 0.8: Measured Central Dist = 337.2109796739634
Longit percentage 0.9: Measured Central Dist = 398.3755219392375


# For Developers

In [20]:
# assert slice mask is equal to intersection with CA1
np.array_equal(np.logical_and(q.raw[...,LON],slice_mask_temp),slice_mask_temp)

True

In [21]:
q.min_max[LON][1] - q.min_max[LON][0]

0.9087291

# Measuring function

The measuring function in coordinae_query.py calculates the positions of the longit endpoints of the slices and from these, euclidian distance is calculated from xyz axes. Hence the warning

In [95]:
center = 0.5
center_down = center - 0.01
center_up = center + 0.01
lon_min,lon_max = slice_boundaries_lon[0],slice_boundaries_lon[1]

idx = q.ltr_range_to_idx(lon_min, lon_max,
                            center_down, center_up,
                            center_down, center_up)
sel_idx = np.argsort(q.raw[tuple(idx.T)], axis=0)[:, 0]
sorted_idx = idx[sel_idx]
xyz = q.coords.indices_to_positions(sorted_idx)[::10, :]
dist_tot = np.sum(np.linalg.norm(np.diff(xyz, axis=0), axis=1))

### Future improvements:

- Add an option for measurements for generation and analysis to be done in geodesic (arc) distance
- Inline visualization of the generated slices

# Generate nrrd masks from lon_min , lon_max data in user.target

- Somehow the serial generation of cylinders did not corresspond to the values given in the user.target . But since we noted their percentage-wise position in LON axis , we can extract the volume information from those

In [15]:
# in case Lmin Lmax data is present for custom slice positions
import re

target_file = '/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/user.target'
with open(target_file,'r') as f:
    f = f.read()
target_data = np.array(re.findall(r'L coordinates: (.*), (.*)\nTarget Cell (.*)',f))
lon_mins,lon_maxs,target_names = target_data[:,0].astype(float),target_data[:,1].astype(float) ,target_data[:,2] 

In [75]:
for lmin,lmax,cur_target in zip(lon_mins,lon_maxs,target_names):
    print(f'{cur_target}: {lmin}-{lmax}')
    slice_mask_from_target = q._get_range_mask(lmin,lmax,LON)
    orientation.with_data(np.asarray(slice_mask_from_target, dtype=np.uint8)).save_nrrd(f"nrrd_masks_in_user_target/{cur_target}.nrrd")

slice0: 0.04726842506877654-0.06694110766895539
slice1: 0.06694110766895539-0.08661379026913424
slice2: 0.08661379026913424-0.10628647286931309
slice3: 0.10628647286931309-0.12595915546949193
slice4: 0.12595915546949193-0.14563183806967078
slice5: 0.14563183806967078-0.16530452066984963
slice6: 0.16530452066984963-0.18497720327002848
slice7: 0.18497720327002845-0.2046498858702073
slice8: 0.2046498858702073-0.22432256847038615
slice9: 0.22432256847038615-0.243995251070565
slice10: 0.24399525107056502-0.26366793367074387
slice11: 0.26366793367074387-0.2833406162709227
slice12: 0.2833406162709227-0.30301329887110157
slice13: 0.3030132988711015-0.32268598147128036
slice14: 0.32268598147128036-0.3423586640714592
slice15: 0.3423586640714592-0.36203134667163805
slice16: 0.36203134667163805-0.3817040292718169
slice17: 0.3817040292718169-0.40137671187199575
slice18: 0.40137671187199575-0.4210493944721746
slice19: 0.4210493944721746-0.44072207707235345
slice20: 0.44072207707235345-0.460394759672